In [1]:
from nbformat import reads, NO_CONVERT
from nbconvert import PythonExporter
def code_extractor(jpt):
    nb = reads(jpt, NO_CONVERT)
    exporter = PythonExporter()
    source, meta = exporter.from_notebook_node(nb)
    return source

In [2]:
import imp
Matchmaker = imp.load_source('name', '../matchmaker.py')
import ast
def function_matching(nb):
    tree = ast.parse(nb)
    mm = Matchmaker.Matchmaker()
    mm.visit(tree)
    matchs = mm.matching()
    return matchs

# Matching the functions 1st version

In [3]:
from pydriller import RepositoryMining
import difflib
import pandas as pd

def repo_matching(repo, commit, path):
    for c in RepositoryMining(repo, single = commit).traverse_commits():
        for modified_files in c.modifications:
            if modified_files.new_path == path:
                try:
                    before = code_extractor(modified_files.source_code_before)
                except TypeError:
                    before = ""
                lines_b = before.strip().splitlines()
                new = code_extractor(modified_files.source_code)
                lines_n = new.strip().splitlines()
                
                diff = difflib.unified_diff(lines_b, lines_n, fromfile='before', tofile='new', lineterm='', n=0)
                lines = list(diff)[2:]
                lineno = 0
                changes = pd.DataFrame(columns = ['repo','path','author','commit','line','code'])
                for line in lines:
                    prefix = '@@'
                    if line.startswith(prefix):
                        s = line[line.find("+"):]
                        try:
                            lineno = int(s[1:s.find(",")])
                        except:
                            lineno = int(s[1:s.find(" ")])
                    else:
                        if line.startswith("+"):
                            changes = changes.append({"repo": c.project_name, 
                                                      "path": modified_files.new_path,
                                                      "author": c.author.email,
                                                      "commit": c.hash,
                                                      "line": lineno,
                                                      "code": line[1:]}, ignore_index= True)
                            lineno += 1
                matches = function_matching(new)
                result = pd.merge(matches, changes, how = 'inner', on= "line")
                return result

# Test

In [ ]:
from pydriller import RepositoryMining, GitRepository
repo = "/home/ubuntu/repos/abulbasar/machine-learning"
for commit in RepositoryMining(repo).traverse_commits():
    for file in commit.modifications:
        print(commit.hash + ' at ' + file.new_path)

In [ ]:
repo = "/home/ubuntu/repos/abulbasar/machine-learning"
path = "01 Neural Network using Numpy.ipynb"
commit = "df41ae028af5a445ec41a94925ac98ad2ccdea01"

m = repo_matching(repo = repo,path = path,commit = commit)


In [ ]:
m.head()

# Running through all commits 1st version

In [ ]:
import pandas as pd
import ast
import csv
# incresing the csv field size
import ctypes
csv.field_size_limit(int(ctypes.c_ulong(-1).value // 2))
failed = []
fields = ['line','function','package','class','repo','path', 'author', 'commit','code']
repo_path = "/home/ubuntu/repos/"
with open('../data/commits_bigquery.csv',"r", encoding="utf8") as csvfile:
    with open("../data/repo_function_matched.csv", "w", encoding="utf8") as newfile:
        data = csv.DictReader(csvfile)
        writer = csv.DictWriter(newfile, fieldnames=fields)
        writer.writeheader()
        for row in data:
            try:
                commit = row['commit']
                repo = row['repo']
                path = row['path']
                path = path.replace("/", "\\")
                repo = repo_path + repo
                matched = repo_matching(repo, commit, path)
                matched.to_csv(newfile, header=False, index= False)
            except KeyboardInterrupt:
                break
            except Exception as e: 
                failed.append(e)
                #print(e)
                #break
import pickle

with open('../data/fails', 'wb') as fp:
    pickle.dump(failed, fp)

--- Logging error ---
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.5/site-packages/nbformat/validator.py", line 258, in validate
    return validator.validate(nbjson)
  File "/home/ubuntu/.local/lib/python3.5/site-packages/jsonschema/validators.py", line 353, in validate
    raise error
jsonschema.exceptions.ValidationError: {'cell_type': 'code', 'metadata': {'collapsed': False}, 'outputs': [{'metadata': {}, 'data': {'text/plain': '<matplotlib.figure.Figure at 0x109443240>', 'image/png': '...\nRU5ErkJggg==\n'}, 'output_type': 'display_data'}], 'source': "all_node_colors = []\nromania_problem = GraphProblem('Arad', 'Bucharest', romania_map)\ndisplay_visual(user_input = False, algorithm = uniform_cost_search, problem = romania_problem)"} is not valid under any of the given schemas

Failed validating 'oneOf' in schema['properties']['cells']['items']:
    {'oneOf': [{'$ref': '#/definitions/raw_cell'},
               {'$ref': '#/definitions/markdown_cell'},
   

--- Logging error ---
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.5/site-packages/nbformat/validator.py", line 258, in validate
    return validator.validate(nbjson)
  File "/home/ubuntu/.local/lib/python3.5/site-packages/jsonschema/validators.py", line 353, in validate
    raise error
jsonschema.exceptions.ValidationError: {'cell_type': 'code', 'execution_count': None, 'metadata': {'collapsed': False}, 'outputs': [{'metadata': {}, 'data': {'text/html': '<style type="text/css">table.blockgrid {border: none;} .blockgrid tr {border: none;} .blockgrid td {padding: 0px;} #blocks5f89d4bf-7cb8-48a7-af12-9acc67eebef5 td {border: 1px solid white;}</style><table id="blocks5f89d4bf-7cb8-48a7-af12-9acc67eebef5" class="blockgrid"><tbody><tr><td title="Index: [0, 0]&#10;Color: (44, 53, 57)" style="width: 20px; height: 20px;background-color: rgb(44, 53, 57);"></td><td title="Index: [0, 1]&#10;Color: (44, 53, 57)" style="width: 20px; height: 20px;background-color: rgb(4

In [ ]:
failed

# Prepare data 2nd version

In [ ]:
import pandas as pd
import numpy as np
import os.path
from os import path

df = pd.read_csv('../data/commits_bigquery.csv')
df = df.drop(df.columns[0], axis = 1)

repos = df.repo.unique()
add_path = np.vectorize(lambda x: "/home/ubuntu/repos/" +x)
repos = add_path(repos)
repos = repos.tolist()
for i in repos:
    if not path.exists(i):
        repos.remove(i)

commits = df.commit.unique().tolist()

paths = df[['commit','path']].groupby('commit')['path'].apply(list)

In [ ]:
len(repos)

# 2nd version

In [ ]:
from pydriller import RepositoryMining
import difflib
import pandas as pd
#inspect all repos and commits
failed = []
fields = ['line','function','package','class','repo','path', 'author', 'commit','code']
newfile = open("../data/repo_function_matched.csv", "w", encoding="utf8")
writer = csv.DictWriter(newfile, fieldnames=fields)
writer.writeheader()
    
for c in RepositoryMining(repos, only_no_merge=True, only_commits=commits).traverse_commits():
    for modified_files in c.modifications:
        try:
            if modified_files.new_path in paths[c.hash]:
                try:
                    before = code_extractor(modified_files.source_code_before)
                except TypeError:
                    before = ""
                lines_b = before.strip().splitlines()
                new = code_extractor(modified_files.source_code)
                lines_n = new.strip().splitlines()
                
                diff = difflib.unified_diff(lines_b, lines_n, fromfile='before', tofile='new', lineterm='', n=0)
                lines = list(diff)[2:]
                lineno = 0
                changes = pd.DataFrame(columns = ['repo','path','author','commit','line','code'])
                for line in lines:
                    prefix = '@@'
                    if line.startswith(prefix):
                        s = line[line.find("+"):]
                        try:
                            lineno = int(s[1:s.find(",")])
                        except:
                            lineno = int(s[1:s.find(" ")])
                    else:
                        if line.startswith("+"):
                            changes = changes.append({"repo": c.project_name, 
                                                      "path": modified_files.new_path,
                                                      "author": c.author.email,
                                                      "commit": c.hash,
                                                      "line": lineno,
                                                      "code": line[1:]}, ignore_index= True)
                            lineno += 1
                matches = function_matching(new)
                result = pd.merge(matches, changes, how = 'inner', on= "line")
                result.to_csv(newfile, header=False, index= False)
        except KeyboardInterrupt:
            break
        except Exception as e: 
            failed.append([c.project_name,modified_files.new_path,e])

newfile.close()
                
import pickle

with open('../data/fails', 'wb') as fp:
    pickle.dump(failed, fp)

# Preparing data 3rd try

In [ ]:
import pandas as pd
import numpy as np
import os.path
from os import path

df = pd.read_csv('../data/commits_bigquery.csv')
df = df.drop(df.columns[0], axis = 1)

repos = df.repo.unique()
add_path = np.vectorize(lambda x: "/home/ubuntu/repos/" +x)
repos = add_path(repos)
repos = repos.tolist()
for i in repos:
    if not path.exists(i):
        repos.remove(i)

commits = df[['repo','commit']].groupby('repo')['commit'].apply(list)
paths = df[['commit','path']].groupby('commit')['path'].apply(list)

# Running 3rd try

In [ ]:
from pydriller import RepositoryMining
import difflib
import pandas as pd
#inspect all repos and commits
failed = []
fields = ['line','function','package','class','repo','path', 'author', 'commit','code']
newfile = open("../data/repo_function_matched.csv", "w", encoding="utf8")
writer = csv.DictWriter(newfile, fieldnames=fields)
writer.writeheader()
    
for r in repos:
    try:
        for c in RepositoryMining(r, only_no_merge=True, only_commits=commits[r]).traverse_commits():
            for f in c.modifications:
                try:
                    if f.new_path in paths[c.hash]:
                        try:
                            before = code_extractor(f.source_code_before)
                        except TypeError:
                            before = ""
                        lines_b = before.strip().splitlines()
                        new = code_extractor(f.source_code)
                        lines_n = new.strip().splitlines()
                        
                        diff = difflib.unified_diff(lines_b, lines_n, fromfile='before', tofile='new', lineterm='', n=0)
                        lines = list(diff)[2:]
                        lineno = 0
                        changes = pd.DataFrame(columns = ['repo','path','author','commit','line','code'])
                        for line in lines:
                            prefix = '@@'
                            if line.startswith(prefix):
                                s = line[line.find("+"):]
                                try:
                                    lineno = int(s[1:s.find(",")])
                                except:
                                    lineno = int(s[1:s.find(" ")])
                            else:
                                if line.startswith("+"):
                                    changes = changes.append({"repo": c.project_name, 
                                                              "path": f.new_path,
                                                              "author": c.author.email,
                                                              "commit": c.hash,
                                                              "line": lineno,
                                                              "code": line[1:]}, ignore_index= True)
                                    lineno += 1
                        matches = function_matching(new)
                        result = pd.merge(matches, changes, how = 'inner', on= "line")
                        result.to_csv(newfile, header=False, index= False)
                except KeyboardInterrupt:
                    break
                except Exception as e: 
                    failed.append(['File',modified_files.new_path,e])
    except KeyboardInterrupt:
        break
    except Exception as e: 
        failed.append(['Project',r,e])

newfile.close()
                
import pickle

with open('../data/fails', 'wb') as fp:
    pickle.dump(failed, fp)